In [1]:
"""This is essentially the same as my denero non-61a review script applied to any 
ratemyprofessors page and any course filter (ex: sahai's page -> 189 reviews,
or sahai's page -> non 189 and non 16a reviews)"""
#import requests 
from bs4 import BeautifulSoup as bs
import re
from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
#PATH = r'C:\\Program Files (x86)\\chromedriver.exe'
PATH = r'C:\Users\jsonn\Documents\Python\selenium\chromedriver.exe' #needed to put r to not get unicode error, this line is from that youtube tutorial
#the path is where I put the chromedriver app thingie

def create_driver(headless=True, implicit_wait=2):
    """
    opts = webdriver.chrome.options.Options()
    opts.headless = headless
    driver = webdriver.Remote(
        command_executor='http://127.0.0.1:3928/wd/hub',
        desired_capabilities=DesiredCapabilities.CHROME, 
        options=opts)
    driver.implicitly_wait(2)
    """
    opts = webdriver.chrome.options.Options()# changed from firefox to chrome 
    opts.headless = headless
    #driver = webdriver.Chrome(options=opts)
    driver = webdriver.Chrome(PATH)
    driver.implicitly_wait(implicit_wait)
    return driver

In [8]:
url = input("paste link: ")
driver = create_driver(headless=False)
driver.get(url)
# Click cookie button
button = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CLASS_NAME, "FullPageModal__StyledCloseButton-sc-17feuxe-1")))
driver.execute_script("arguments[0].click();", button)

try:
    while True:
        button = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.CLASS_NAME, "Buttons__BlackButton-sc-19xdot-1")))
        driver.execute_script("arguments[0].click();", button)
except:
    pass

professor_soup = bs(driver.page_source, "html.parser")
driver.quit()

paste link: https://www.ratemyprofessors.com/ShowRatings.jsp?tid=2369550


In [9]:
filter_type = input("Choose the kind of filter you want. If you want a specific class(es), enter course number(s) (i.e. CS61A) one at a time, and enter 'done' when complete. If you want to filter OUT a course(s), just press enter for now: ")

if filter_type: #they need to put in the whole course name, and potentially other names that it might go by
    courses_wanted = [filter_type.upper()]
    while filter_type != "done":
        filter_type = input("choose another course. Enter 'done' when complete: ")
        courses_wanted.append(filter_type.upper())
    courses_wanted.pop() #gets rid of done, this doesn't really matter unless there's actually a course called done
print('Execute next cell.')

Choose the kind of filter you want. If you want a specific class(es), enter course number(s) (i.e. CS61A) one at a time, and enter 'done' when complete. If you want to filter OUT a course(s), just press enter for now: 
Execute next cell.


In [10]:
course_names = professor_soup.find_all("div","RatingHeader__StyledClass-sc-1dlkqw1-2 hBbYdP")
names = [name.string for name in course_names][::2]
course_reviews = professor_soup.find_all("div", "Comments__StyledComments-dzzyvm-0 dEfjGB")
reviews = [review.string for review in course_reviews][1:] #we don't want most helpful rating because it's repeated 

#print(len(names), len(reviews)) i need to figure out why reviews is half the length of names 
#the length of names is resolved. the source code for some reason had stuff listed twice

course_ratings_difficulty = professor_soup.find_all("div", re.compile("RatingValues__RatingValue-sc-6dc747-3"))
#ratingvalues with 747-3 are course rating and difficulty
ratings, difficulties = [rating.string for rating in course_ratings_difficulty[::2]], [difficulty.string for difficulty in course_ratings_difficulty[1::2]]
print("Execute next cell.")

Execute next cell.


In [11]:
divs_soup = professor_soup.find_all("div", "CourseMeta__StyledCourseMeta-x344ms-0")
divs = [div.text for div in divs_soup]
grades_list = []
grading_scale = {4.0:'A',
                    3.7: 'A-',
                    3.3: 'B+',
                     3.0: 'B',
                    2.7: 'B-',
                    2.3: 'C+',
                    2: 'C',
                    1.7:'C-',
                    1.3:'D+',
                     1:'D',
                     0.7:'D-',
                    0:'F'}
reversed_scale = {letter:num for num, letter in grading_scale.items()}

def average_grade(grades):
    """given a list of grades as numbers (i.e. A is a 4) returns the average grade as a letter.
    This function rounds down.
    average_grade([4.3,2.3,4,3.7,3.3]), where the mean is 3.52, rounds up to an A-
    average_grade([4,3,3,4,4,2.7]), where the mean is 3.449, rounds down to a b+"""
    number_grade = round(sum(grades) / len(grades), 1)
    for grade in grading_scale:
        if number_grade >= grade -.2: #starts with a, so if the average grade is >= 3.8
            return grading_scale[grade]
        
def grade_to_number(letter):
    if letter == "A+": #this is not really right but it's fine, there's no difference between the two
        letter = "A"
    return reversed_scale[letter]

"""make grades list"""
def grade_isolator(line, grades):
    if "Grade" in line:
        parts = line.split("Grade: ") 
        moreparts = parts[1].split("Text")
        grades.append(moreparts[0])
    else:
        grades.append('Z')  
        
"""get average grade"""
def avg(grades):
    glist = []
    for grade in grades:
        if grade in grading_scale.values() or grade == "A+":
            glist.append(grade_to_number(grade))
    return average_grade(glist) if glist else 0 #if there were no grades 


In [12]:
average_rating, average_difficulty, index, ratings_list, difficulties_list = 0, 0, 0, [], [] #by course wanted for if filter_type true
ratings_count = {x:0 for x in range(1,6)}
difficulty_count = {x:0 for x in range(1,6)}

if filter_type: #because the other option was "" #some implicit booleanness
    for name, review, rating, difficulty in zip(names, reviews, ratings, difficulties):
        print(name)
        if any([name in course for course in courses_wanted]):
     #   if name in courses_wanted:# or name[len(name)-3:] in [course[len(course)-3:] for course in courses_wanted]:# or name[len(name)-2:] in [course[len(course)-2:] for course in courses_wanted]:
            print(review, rating, difficulty, '\n')
            ratings_list.append(rating)
            difficulties_list.append(difficulty)
            if float(rating) in ratings_count:
                ratings_count[float(rating)] +=1
            else:
                ratings_count[float(rating)] = 1
            if float(difficulty) in difficulty_count:
                difficulty_count[float(difficulty)] +=1
            else:
                difficulty_count[float(difficulty)] = 1
            average_rating += float(rating)
            average_difficulty += float(difficulty)
            index +=1
    for index, div in enumerate(divs):
        if any([names[index] in course for course in courses_wanted]):
            grade_isolator(div, grades_list)
else: #for filtering out courses
    course, filtered = input("enter the course number(s) of the classes that you would NOT like to see. Press enter on blank when you're complete: "), []
    while course:
        filtered.append(course.upper())
        course = input("enter the course number(s) of the classes that you would NOT like to see. Press enter on blank when you're complete: " )
    for name, review, rating, difficulty in zip(names, reviews, ratings, difficulties):
        print(name)
        if not any([name in course for course in filtered]):# i.e. class should NOT be CS61A filtered
            print(review, rating, difficulty, '\n')
            ratings_list.append(rating)
            difficulties_list.append(difficulty)
            if float(rating) in ratings_count:# or difficulty in difficulty_count:
                ratings_count[float(rating)] +=1
            else:
                ratings_count[float(rating)] = 1
            if float(difficulty) in difficulty_count:
                difficulty_count[float(difficulty)] +=1
            else:
                difficulty_count[float(difficulty)] = 1
            average_rating += float(rating)
            average_difficulty += float(difficulty)
            index +=1   
    for index, div in enumerate(divs):
        if not any([names[index] in course for course in filtered]):
            grade_isolator(div, grades_list)
try:
    print("Average rating under filters: {} \nAverage difficulty under filters: {}".format(average_rating/index, average_difficulty/index))
    print("Max/min rating under filters: {}, {} \nMax/min difficulty under filters: {}, {}".format(max(ratings_list), min(ratings_list), max(difficulties_list), min(difficulties_list)))
except ZeroDivisionError:
    pass
print("Average grade: {}".format(avg(grades_list)))

enter the course number(s) of the classes that you would NOT like to see. Press enter on blank when you're complete: 
EECS127
Gireeja is one of the most caring professors I have met at Berkeley. However, she ought to prepare more for her lectures cause she does make quite a few mistakes that can be distracting and derail lecture for several minutes. I also wish she would be more transparent with her grading scheme. 3.0 3.0 

EECS127
Crystal clear lectures, fair tests, the right amount of homework. Overall a great class and she does a great job of teaching it. 5.0 4.0 

EE16A
Ranade is super eloquent, clear, and concise. She knows what she's talking about and is always ready to answer peoples questions. I think she's also encouraging. Sometimes lectures would get pretty boring, and I would recommend watching webcast if you prefer fast paced, as she speaks slowly (probably because this class is meant for freshmen). 5.0 4.0 

EECS16A
Amazing and very clear lectures! Prof Ranade is willing

In [13]:
gender = {0:0, 1:0 } #0 female, 1 male
she_series, he_series = ('she','her','hers'),("he","him","his")
for review in [review.lower() for review in reviews]:
    for word in review.split():
        if word in she_series:
            #print(word)
            gender[0] +=1
        if word in he_series:
            #print(word)
            gender[1] +=1
if gender[0] == gender[1]:
    print("gender inconclusive")
else:
    reverse = {val: key for key, val in gender.items()}
    print("gender: {}".format(reverse[max(reverse.keys())]))


she
her
she
she
her
she
she
she
she
she
her
her
her
she
her
gender: 0


In [ ]:
"""general cell for debugging and trying stuff out"""
print(len(names), len(reviews), len(ratings), len(difficulties), len(divs))
print(sum([val for val in ratings_count.values()]))
print(sum(difficulty_count.values()))
print(ratings_count, difficulty_count)
def filter_by_grades(thing):
    return any([s.lower() in ["a+", "b-","b+" ] for s in thing.split()]) 
letterz,gradesss = professor_soup.find_all(string = filter_by_grades), []
for letter in letterz:
    gradesss.append(grade_to_number(letter))